In [1]:
#Environment set-up and libraries

#Base libraries
import numpy as np
import random
import torch
import torch.nn as nn

#Plotting libraries
%matplotlib inline
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go

#Utilities libraries
from glob import glob 
import os

import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_file(file_name):
    print(file_name)

    if file_name.endswith(".las") or file_name.endswith(".laz"):
        print("[INFO] .las (.laz) file loading")
        try:
            # import lidar .las data and assign to variable
            pcd = laspy.read(file_name)
            # examine the available features for the lidar file we have read
            # list(las.point_format.dimension_names)
            #
            # set(list(las.classification))

            # Creating, Filtering, and Writing Point Cloud Data
            # To create 3D point cloud data, we can stack together with the X, Y, and Z dimensions, using Numpy like this.
            point_data = np.stack([pcd.X, pcd.Y, pcd.Z], axis=0).transpose((1, 0))
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_data)
            # points = point_data
            if pcd is not None:
                print("[Info] Successfully read", file_name)

                # Point cloud
                return pcd

        except Exception:
            print(".las, .laz file load failed")

    elif file_name.endswith(".e57"):
        print("[INFO] .e57 file loading")
        try:
            e57_file = pye57.E57(file_name)

            # other attributes can be read using:
            data = e57_file.read_scan(0)

            # 'data' is a dictionary with the point types as keys
            # assert isinstance(data["cartesianX"], np.ndarray)
            # assert isinstance(data["cartesianY"], np.ndarray)
            # assert isinstance(data["cartesianZ"], np.ndarray)

            point_xyz = np.stack([data["cartesianX"], data["cartesianY"], data["cartesianZ"]]).transpose((1, 0))
            # points_rgb = [data["colorRed"], data["colorGreen"], data["colorBlue"]]
            # points_intensity = data["intensity"]

            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            # points = o3d.utility.Vector3dVector(point_xyz)
            # points = point_xyz
            # pcd.colors = o3d.utility.Vector3dVector(points_rgb)
            # pcd.colors[:, 0] = points_intensity
            print("[Info] Successfully read", file_name)
            return pcd

        except Exception:
            print(".e57 file load failed")

    elif file_name.endswith(".bin"):
        print("[INFO] .bin file loading")
        try:
            size_float = 4
            list_pcd = []
            with open(file_name, "rb") as f:
                byte = f.read(size_float * 4)
                while byte:
                    x, y, z, intensity = struct.unpack("ffff", byte)
                    list_pcd.append([x, y, z])
                    byte = f.read(size_float * 4)
            np_pcd = np.asarray(list_pcd)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(np_pcd)
            print("[Info] Successfully read", file_name)
            return pcd

        except Exception:
            print(".bin file load failed")

    elif file_name.endswith(".ply"):
        pcd = o3d.io.read_point_cloud(file_name)
        points_xyz = np.asarray(pcd.points)
        #pcd = o3d.geometry.PointCloud() # No need to do that already a PointCloud
        pcd.points = o3d.utility.Vector3dVector(points_xyz)
        # points = points_xyz
        if pcd is not None:
            print("[Info] Successfully read", file_name)
            # Point cloud
            return pcd

    elif file_name.endswith(".pts"):
        try:
            with open(file_name, "r") as f:
                # Log every 1000000 lines.
                LOG_EVERY_N = 1000000
                points_np = []
                for line in f:
                    if len(line.split()) == 4:
                        x, y, z, i = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 3:
                        x, y, z = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 5:
                        x, y, z, i, zeroes_v = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z), float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    elif len(line.split()) == 7:
                        x, y, z, r, g, b, i = [num for num in line.split()]
                        points_np.append([float(x), float(y), float(z),
                                          float(r), float(g), float(b),
                                          float(i)])
                        if (len(points_np) % LOG_EVERY_N) == 0:
                            print('point', len(points_np))
                    else:
                        print("[Info] The file has unregistered format")
                        return
            print('loop end')
            points_arr = np.array(points_np).transpose()
            print(len(points_arr))
            point_xyz = points_arr[:3].transpose()
            print("xyz points shape", point_xyz.shape)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(point_xyz)
            if len(points_arr) == 4:
                points_intensity = (points_arr[3])/255.0
                print("intensity points len", points_intensity.shape)
                points_intensity_rgb = np.vstack((points_intensity,
                                                  points_intensity,
                                                  points_intensity)).T
                print("intensity_rgb points shape", points_intensity_rgb.shape)
                pcd.colors = o3d.utility.Vector3dVector(points_intensity_rgb)
            elif len(points_arr) == 7:
                points_red = (points_arr[4]) / 255.0
                points_green = (points_arr[5]) / 255.0
                points_blue = (points_arr[6]) / 255.0
                points_rgb = np.vstack((points_red,
                                        points_green,
                                        points_blue)).T

                # points_intensity = ((points_arr[3]) / 255.0).T
                # print("intensity points len", points_intensity.shape)
                print("rgb points shape", points_rgb.shape)
                pcd.colors = o3d.utility.Vector3dVector(points_rgb)
                #pcd.intensities = o3d.utility.Vector3dVector(points_intensity)
            if pcd is not None:
                print("[Info] Successfully read", file_name)
                # Point cloud
                return pcd

        except Exception:
            print("[Info] Reading .pts file failed", file_name)

    # elif file_name.endswith(".kml"):
    #     try:
    #         with open(file_name, "r") as f:
    #             # Log every 1000000 lines.
    #             LOG_EVERY_N = 1000000
    #             points_np = []
    #             for line in f:
    #                 print(line)
    #                 if len(line.split(",")) == 3 and (line[0].isdigit() or line.startswith("-")):
    #                     y, x, z = [num for num in line.split(",")]
    #                     points_np.append([float(x), float(y), float(z)])
    #                     if (len(points_np) % LOG_EVERY_N) == 0:
    #                         print('point', len(points_np))
    #                 else:
    #                     print("[Info] The file has unregistered format")
    #         print('loop end')
    #         points_arr = np.array(points_np).transpose()
    #         print(len(points_arr))
    #         point_xyz = points_arr[:3].transpose()
    #         # points_intensity = points_arr[3]
    #         pcd = o3d.geometry.PointCloud()
    #         pcd.points = o3d.utility.Vector3dVector(point_xyz)
    #         if pcd is not None:
    #             print("[Info] Successfully read", file_name)
    #             # Point cloud
    #             return pcd
    #
    #     except Exception:
    #         print("[Info] Reading .kml file failed", file_name)

    else:
        pcd = None
        geometry_type = o3d.io.read_file_geometry_type(file_name)
        print(geometry_type)

        mesh = None
        if geometry_type & o3d.io.CONTAINS_TRIANGLES:
            mesh = o3d.io.read_triangle_model(file_name)
        if mesh is None:
            print("[Info]", file_name, "appears to be a point cloud")
            cloud = None
            try:
                cloud = o3d.io.read_point_cloud(file_name)
                # print(type(cloud))
            except Exception:
                print("[Info] Unknown filename", file_name)
            if cloud is not None:
                print("[Info] Successfully read", file_name)

                if not cloud.has_normals():
                    cloud.estimate_normals()
                cloud.normalize_normals()
                pcd = cloud
                #points = cloud.points
                pcd.points = o3d.utility.Vector3dVector(cloud.points)
            else:
                print("[WARNING] Failed to read points", file_name)

        if pcd is not None or mesh is not None:
            try:
                if mesh is not None:
                    # Triangle model
                    _scene.scene.add_model("__model__", mesh)
                else:
                    # Point cloud
                    return pcd

            except Exception as e:
                print(e)


In [3]:
filepath_mob1 = "/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/"
filename1 = "SABRE MMS_S3 - 0002.pts"
filepath1 = filepath_mob1 + filename1
pc1 = load_file(filepath1)

filepath_static2 = "/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE - Selected Static Scan Data/"
filename2 = "SABRE Static Scan_T17_003.pts"
filepath2 = filepath_static2 + filename2
pc2 = load_file(filepath2)

/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/SABRE MMS_S3 - 0002.pts
point 1000000
point 2000000
point 3000000
point 4000000
point 5000000
point 6000000
point 7000000
point 8000000
point 9000000
point 10000000
point 11000000
loop end
4
xyz points shape (11008825, 3)
intensity points len (11008825,)
intensity_rgb points shape (11008825, 3)
[Info] Successfully read /home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE ADVANCED 3D - Selected MMS Data/SABRE MMS_S3 - 0002.pts
/home/mekala/PycharmProjects/SabreProject_code/Sabre_proj/SABRE - Selected Static Scan Data/SABRE - Selected Static Scan Data/SABRE Static Scan_T17_003.pts
point 1000000
point 2000000
point 3000000
point 4000000
point 5000000
point 6000000
point 7000000
point 8000000
point 9000000
point 10000000
point 11000000
point 12000000
point 13000000
point 14000000
point 15000000
point 16000000
point 17

In [22]:
def create_voxel_boxes(point_cloud, num_voxels):
    """
    Create voxel boxes for a given point cloud and number of voxels along each axis.

    Parameters:
    - point_cloud: Nx3 numpy array representing the point cloud.
    - num_voxels: List or tuple with the number of voxels along each axis (x, y, z).

    Returns:
    - voxel_boxes: List of Open3D OrientedBoundingBox objects.
    """
    
    # Compute the bounding box of the point cloud
    min_bound = np.min(point_cloud, axis=0)
    max_bound = np.max(point_cloud, axis=0)

    # Determine the voxel size in each dimension
    voxel_size = (max_bound - min_bound) / num_voxels

    voxel_boxes = []

    for i in range(num_voxels[0]):
        for j in range(num_voxels[1]):
            for k in range(num_voxels[2]):
                # Compute the center of the voxel
                center = min_bound + voxel_size * (np.array([i, j, k]) + 0.5)
                # Create an oriented bounding box for the voxel
                obb = o3d.geometry.OrientedBoundingBox(center=center, R=np.eye(3), extent=voxel_size)
                voxel_boxes.append(obb)
                
    voxel_dict = {}

    for point in point_cloud:
        # Determine the voxel index for the point
        voxel_index = tuple(((point - min_bound) // voxel_size).astype(int))
        
        if voxel_index not in voxel_dict:
            voxel_dict[voxel_index] = []
        
        voxel_dict[voxel_index].append(point)

    return voxel_boxes, voxel_dict

In [23]:
def visualize_point_cloud_with_voxels(point_cloud, voxel_boxes):
    """
    Visualize a point cloud alongside voxel boxes.

    Parameters:
    - point_cloud: Nx3 numpy array representing the point cloud.
    - voxel_boxes: List of Open3D OrientedBoundingBox objects.
    """
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    print("Number of voxels:", len(voxel_boxes))

    o3d.visualization.draw_geometries([pcd, *voxel_boxes])
    
#     #Visualize
#     o3d.visualization.draw_plotly([pcd, *voxel_boxes],point_sample_factor=0.5, width=600, height=400)
#     visualizer = JVisualizer()
#     visualizer.add_geometry(pcd)
#     visualizer.add_geometry(*voxel_boxes)
#     visualizer.show()
    
    #draw(pcd)


In [24]:
pc1_points = np.asarray(pc1.points)
pc2_points = np.asarray(pc2.points)
print(pc1_points.shape)

num_voxels = [10, 20, 1]
voxel_boxes, voxel_dict = create_voxel_boxes(pc1_points, num_voxels)

# Count the number of points in each voxel
for voxel_index, points in voxel_dict.items():
    print(f"Voxel {voxel_index} contains {len(points)} points.")
    
visualize_point_cloud_with_voxels(pc1_points, voxel_boxes)

(11008825, 3)
Voxel (9, 9, 0) contains 241619 points.
Voxel (9, 8, 0) contains 869253 points.
Voxel (9, 10, 0) contains 1472 points.
Voxel (9, 7, 0) contains 23793 points.
Voxel (9, 2, 0) contains 169 points.
Voxel (9, 5, 0) contains 206 points.
Voxel (9, 6, 0) contains 1137 points.
Voxel (9, 4, 0) contains 93 points.
Voxel (9, 3, 0) contains 1 points.
Voxel (8, 9, 0) contains 145105 points.
Voxel (8, 10, 0) contains 13920 points.
Voxel (8, 8, 0) contains 968397 points.
Voxel (8, 7, 0) contains 21659 points.
Voxel (8, 4, 0) contains 37 points.
Voxel (8, 5, 0) contains 157 points.
Voxel (8, 6, 0) contains 1798 points.
Voxel (8, 2, 0) contains 1 points.
Voxel (8, 12, 0) contains 611 points.
Voxel (8, 11, 0) contains 3980 points.
Voxel (8, 18, 0) contains 4 points.
Voxel (8, 1, 0) contains 26 points.
Voxel (8, 3, 0) contains 1 points.
Voxel (8, 14, 0) contains 52 points.
Voxel (8, 13, 0) contains 809 points.
Voxel (7, 17, 0) contains 4 points.
Voxel (7, 14, 0) contains 2161 points.
Voxel 

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to

In [9]:
def compute_chamfer_distance(pcd1, pcd2):
    """
    Compute the Chamfer distance between two point clouds.

    Parameters:
    - pcd1, pcd2: Open3D point cloud objects.

    Returns:
    - chamfer_distance: The Chamfer distance between the two point clouds.
    """
    
    # Compute distance from pcd1 to pcd2
    distances_1_to_2 = pcd1.compute_point_cloud_distance(pcd2)
    avg_distance_1_to_2 = np.mean([np.min(dist) for dist in distances_1_to_2])

    # Compute distance from pcd2 to pcd1
    distances_2_to_1 = pcd2.compute_point_cloud_distance(pcd1)
    avg_distance_2_to_1 = np.mean([np.min(dist) for dist in distances_2_to_1])

    # Compute the Chamfer distance
    chamfer_distance = (avg_distance_1_to_2 + avg_distance_2_to_1) / 2

    return chamfer_distance

In [14]:
points1_mean = pc1_points.mean(axis=0)
points2_mean = pc2_points.mean(axis=0)
points1 = (points1.T - points1_mean).T
points2 = (points2.T - points2_mean).T
print(points1, points2)

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(points1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(points2)

chamfer_dist = compute_chamfer_distance(pcd1, pcd2)
print(f"Chamfer Distance: {chamfer_dist}")

[[-5.31879285e+05 -5.31878285e+05 -5.31877285e+05]
 [-6.32451532e+06 -6.32451432e+06 -6.32451332e+06]
 [-1.14778355e+02 -1.13778355e+02 -1.12778355e+02]] [[-3.71352031e+05 -3.71351031e+05 -3.71350031e+05]
 [-7.96968610e+05 -7.96967610e+05 -7.96966610e+05]
 [-6.49070194e+01 -6.39070194e+01 -6.29070194e+01]]
Chamfer Distance: 1764899.2348312219


In [ ]:
class PointCloudRegistration:
    def __init__(self):
        pass

    def closest_point(self, source, target):
        """
        For each point in source, find its closest point in target.
        """
        dists = torch.cdist(source, target)
        mins, idxs = torch.min(dists, dim=1)
        return target[idxs]

    def compute_transformation(self, source, target):
        """
        Compute the R, T transformation to align source to target.
        """
        # Compute centroids
        centroid_source = torch.mean(source, dim=0)
        centroid_target = torch.mean(target, dim=0)

        # Center the point clouds
        source_centered = source - centroid_source
        target_centered = target - centroid_target

        # Compute the covariance matrix
        H = torch.mm(source_centered.t(), target_centered)

        # Singular Value Decomposition
        U, S, V = torch.svd(H)

        # Compute the rotation matrix
        R = torch.mm(V, U.t())

        # Ensure the rotation matrix is right-handed
        if torch.det(R) < 0:
            V[:, -1] *= -1
            R = torch.mm(V, U.t())

        # Compute the translation vector
        T = centroid_target - torch.mm(centroid_source.unsqueeze(0), R)

        return R, T

    def register(self, source, target, max_iterations=100, tolerance=1e-5):
        """
        Aligns the source point cloud to the target point cloud using ICP.
        """
        prev_error = 0

        for i in range(max_iterations):
            # Find the closest points
            closest = self.closest_point(source, target)

            # Compute the transformation
            R, T = self.compute_transformation(source, closest)

            # Apply the transformation
            source = torch.mm(source, R.t()) + T

            # Compute the mean squared error
            error = torch.mean(torch.norm(source - closest, dim=1))

            # Check for convergence
            if torch.abs(prev_error - error) < tolerance:
                break

            prev_error = error

        return source

In [ ]:
reg = PointCloudRegistration()
aligned_source = reg.register(source, target)

print(aligned_source)